In [1]:
import json
import random
import pickle
import numpy as np
from tensorflow.keras.models import load_model
import nltk
from nltk.stem import WordNetLemmatizer


In [2]:
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))
model = load_model('chatbot.h5')
lemmatizer = WordNetLemmatizer()

In [3]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def bow(sentence, words, show_details=True):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))
def predict_class(sentence, model):
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [4]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result
def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

In [6]:
import pyttsx3
import speech_recognition as sr
import wikipedia
import flask
import webbrowser
from flask import Flask, render_template,url_for,request

def talk(string):
    engine = pyttsx3.init()
    engine.say(string)
    engine.runAndWait()
while True:
    msg = input("You: ")
    assistance = chatbot_response(msg)
    print(assistance)
    talk(assistance)
        
    if 'quit' in msg.lower():
        break

You: hey
Hello, How can I be of service to you
You: I am looking for a rental car
Are you looking to rent today or later this week?
You: what cars do you have
We rent Yamaha, Piaggio and Vespa mopeds
You: I need yamaha today
Same-day rentals please call 1-800-MYMOPED
You: thank you
Happy to help!
You: quit
Have a nice day


In [40]:
text = [']
talk = pyttsx3.init()
talk.say(text)
talk.runAndWait()